# Etude des dépendances en températures entre deux positions géographiques

In [13]:
import pandas as pd
from datetime import datetime
import numpy as np

In [10]:
# Suppression des relevés pour les 29/02 des années bissextiles
path = "Data/ukcp09_gridded-land-obs-daily_timeseries_mean-temperature_000000E_500000N_19600101-20161231.csv"
data = pd.read_csv(path, header=[0,1], index_col=0, parse_dates=True)
dates_to_remove = [datetime.date(datetime.strptime(str(y)+"-02-29", '%Y-%m-%d')) for y in range(1960, 2017, 4)]
data = data.drop(dates_to_remove)
# Lecture de la série des résidus (enregistrée indépendemment)
serie1 = pd.read_csv("BB22500-547500.csv", header=None)[1]
serie1.index = data.index
serie2 = pd.read_csv("BB17500-502500.csv", header=None)[1]
serie2.index = data.index
serie2.head()

1960-01-01    2.525530
1960-01-02    0.784470
1960-01-03    5.845865
1960-01-04    4.523752
1960-01-05   -0.348186
Name: 1, dtype: float64

In [14]:
# On ne garde qu'une valeur sur 5
serie1 = serie1.values[list(range(0,len(serie1),5))]
serie2 = serie2.values[list(range(0,len(serie2),5))]

# Corrélation linéaire

In [19]:
correlation = (np.mean(serie1 * serie2) - np.mean(serie1)*np.mean(serie2))/(np.std(serie1)*np.std(serie2))
print("Correlation : "+str(correlation))

Correlation : 0.9871961704389657


# Corrélation des rangs

In [29]:
# Coefficient de Spearman
n = len(serie1)
argsort1 = np.argsort(serie1)
ranks1 = {}
for k in range(len(argsort1)):
    ranks1[serie1[argsort1[k]]] = k
    
argsort2 = np.argsort(serie2)
ranks2 = {}
for k in range(len(argsort2)):
    ranks2[serie2[argsort2[k]]] = k

In [33]:
tmp1 = np.array([ranks1[x] for x in serie1])
tmp2 = np.array([ranks2[x] for x in serie2])
rho_s = 12/(n**2-1)*1/n*np.sum((tmp1-(n+1)/2)*(tmp2-(n+1)/2))
print("Coefficient de Spearman : "+str(rho_s))

Coefficient de Spearman : 0.9860487621683954


In [39]:
# Taux de Kendall
t = 0
for i in range(n):
    for j in range(i,n):
        t += np.sign((serie1[i]-serie1[j])*(serie2[i]-serie2[j]))
t /= n*(n-1)/2
print("Taux de Kendall : "+str(t))

Taux de Kendall : 0.9034454550496367


Tous ces indicateurs témoignent d'une corrélation très forte entre les deux séries temporelles

# Copules

## Copule de Gumbel

In [48]:
# Estimation par le taux de Kendall
theta_g = 1/(1-t)
print("Theta (copule de Gumbel) : "+str(theta_g))

Theta (copule de Gumbel) : 10.356840276281964


In [49]:
# Indépendance asymptotique à gauche
lbda_u = 2-2**(1/theta_g)
print("Coefficient de dépendance extrême à droite : " + str(lbda_u))

Coefficient de dépendance extrême à droite : 0.9307831008183443


In [ ]:
# Par maximum de vraisemblance

## Copule de Clayton

In [46]:
# Estimation par le taux de Kendall
theta_c = 2*t/(1-t)
print("Theta (copule de Clayton) : "+str(theta_c))

Theta (copule de Clayton) : 18.71368055256393


In [50]:
# Indépendance asymptotique à droite
lbda_l = 2**(-1/theta_c)
print("Coefficient de dépendance extrême à gauche : " + str(lbda_l))

Coefficient de dépendance extrême à gauche : 0.9636379777034911


In [51]:
# Estimation par maximum de vraisemblance